### **<span style="color:green">VISÃO GERAL DA BASE DE DADOS: Titanic</span>**

Esta base de dados contém informações sobre os passageiros do Titanic, o famoso navio que afundou em 1912. Ela é amplamente utilizada para análises de sobrevivência e para entender os fatores que influenciaram a sobrevivência dos passageiros. Aqui estão algumas das variáveis incluídas:

- **PassengerId**: Identificador único de cada passageiro.
- **Survived**: Indica se o passageiro sobreviveu (1) ou não (0).
- **Pclass**: Classe do bilhete do passageiro (1ª, 2ª, 3ª).
- **Name**: Nome do passageiro.
- **Sex**: Sexo do passageiro.
- **Age**: Idade do passageiro.
- **SibSp**: Número de irmãos/cônjuges a bordo do Titanic.
- **Parch**: Número de pais/filhos a bordo do Titanic.
- **Ticket**: Número do bilhete.
- **Fare**: Tarifa paga pelo bilhete.
- **Cabin**: Número da cabine.
- **Embarked**: Porto de embarque (C = Cherbourg; Q = Queenstown; S = Southampton).

A meta é entender como diferentes características dos passageiros e suas condições de viagem podem influenciar a probabilidade de sobrevivência no naufrágio do Titanic. Essa base de dados pode ser usada para diversas análises, como identificar padrões de sobrevivência, visualizar tendências demográficas dos passageiros e realizar previsões sobre fatores que afetaram as chances de sobrevivência.

Link para a competição do Kaggle: https://www.kaggle.com/competitions/titanic/overview

---

### **<span style="color:green">IMPORTS E CONFIGURAÇÕES</span>**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, roc_curve, accuracy_score, precision_score, recall_score, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

#!pip install ipywidgets

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import (
    LogisticRegression,
    RidgeClassifier,
    SGDClassifier,
)

from sklearn.ensemble import (
    AdaBoostClassifier, 
    BaggingClassifier, 
    ExtraTreesClassifier, 
    RandomForestClassifier, 
    StackingClassifier, 
    GradientBoostingClassifier, 
    HistGradientBoostingClassifier,
    VotingClassifier
)

from sklearn.preprocessing import StandardScaler

### **<span style="color:green">FUNÇÕES DE PROCESSAMENTO DE DADOS</span>**

In [ ]:
CAMINHO_BASE_TREINAMENTO = 'https://raw.githubusercontent.com/claytonsilva007/IDP/refs/heads/main/ml-01/dados/titanic/train.csv'
CAMINHO_BASE_TESTE = 'https://raw.githubusercontent.com/claytonsilva007/IDP/refs/heads/main/ml-01/dados/titanic/test.csv'

**Descrição**:
A função `carregar_dados` tem como objetivo carregar um arquivo CSV a partir de um caminho especificado e retornar os dados em um DataFrame do pandas.

**Parâmetros**:
- `path` (str): O caminho para o arquivo CSV que será carregado.

**Retorno**:
- Retorna um DataFrame contendo os dados do arquivo CSV.

**Funcionamento**:
1. A função recebe um caminho (`path`) como string, que especifica a localização do arquivo CSV.
2. Utiliza a função `pd.read_csv` do pandas para ler o arquivo CSV e armazenar os dados em um DataFrame.
3. Retorna o DataFrame contendo os dados carregados.


In [ ]:
def carregar_dados(path: str):
  dados = pd.read_csv(path)
  return dados

**Descrição**:
A função `obter_treino_e_teste` divide um DataFrame em conjuntos de treino e teste com base em uma porcentagem especificada para o treino.

**Parâmetros**:
- `df` (pd.DataFrame): O DataFrame contendo os dados a serem divididos.
- `perc_treino` (float, opcional): A proporção dos dados a serem usados para o treino. O valor padrão é 0.8 (80%).

**Retorno**:
- Retorna duas partes do DataFrame: `treino` e `teste`.

**Funcionamento**:
1. A função recebe um DataFrame (`df`) e uma proporção (`perc_treino`) para o conjunto de treino.
2. Utiliza a função `train_test_split` para dividir o DataFrame em conjuntos de treino e teste com a proporção especificada. A divisão é feita de forma aleatória, mas reprodutível devido ao `random_state=42`.
3. Retorna dois DataFrames: `treino` contendo a parte de treino e `teste` contendo a parte de teste.

In [ ]:
def obter_treino_e_teste(df: pd.DataFrame, perc_treino: float = 0.8):
    treino, teste = train_test_split(df, train_size=perc_treino ,random_state=42)
    return treino, teste  

**Descrição**:
A função `transformar_sex_em_inteiro` converte a coluna 'Gender' de um DataFrame, transformando os valores categóricos 'Male' e 'Female' em valores inteiros.

**Parâmetros**:
- `df` (pd.DataFrame): O DataFrame contendo a coluna 'Gender' que será transformada.

**Retorno**:
- Retorna o DataFrame com a coluna 'Gender' transformada em valores inteiros.

**Funcionamento**:
1. A função recebe um DataFrame (`df`) que contém a coluna 'Gender'.
2. Utiliza o método `map` para converter os valores 'Male' e 'Female' em 0 e 1, respectivamente.
3. Retorna o DataFrame com a coluna 'Gender' atualizada.

In [ ]:
def transformar_sex_em_inteiro(df: pd.DataFrame):
    df.Sex = df.Sex.map({'male': 0, 'female': 1})
    return df

**Descrição**:
A função `adicionar_indicador_menor_idade` adiciona uma nova coluna chamada 'isChild' a um DataFrame. Esta coluna indica se a idade (coluna 'Age') é menor que 12 anos.

**Parâmetros**:
- `df` (pd.DataFrame): O DataFrame contendo a coluna 'Age' que será utilizada para criar o indicador.

**Retorno**:
- Retorna o DataFrame com a nova coluna 'isChild', onde o valor é 1 se a idade é menor que 18 anos e 0 caso contrário.

**Funcionamento**:
1. A função recebe um DataFrame (`df`) que contém a coluna 'Age'.
2. Utiliza o método `map` com uma função `lambda` para criar a coluna 'isChild':
    - Se a idade for menor que 12 (`age < 12`), a coluna 'isChild' recebe o valor 1.
    - Caso contrário, a coluna 'isChild' recebe o valor 0.
3. Retorna o DataFrame com a nova coluna 'isChild' adicionada.

In [ ]:
def adicionar_indicador_menor_idade(df: pd.DataFrame):
    df['isChild'] = df.Age.map(lambda age: 1 if age < 12 else 0)
    return df

**Descrição**:
A função `get_pronouns` extrai os pronomes de tratamento dos nomes dos passageiros do Titanic, presentes em uma string formatada.

**Parâmetros**:
- `name` (str): Uma string contendo o nome completo do passageiro, com o sobrenome seguido por um título e o primeiro nome (ex: "Smith, Mr. John").

**Retorno**:
- Retorna uma string contendo o pronome de tratamento extraído do nome. Caso o formato do nome não contenha um pronome de tratamento, a função retorna "None".

**Funcionamento**:
1. A função recebe uma string (`name`) que contém o nome completo do passageiro.
2. Verifica se a string contém uma vírgula (`, `) e um ponto (`.`).
3. Se ambos estão presentes, divide a string em dois passos:
    - Primeiramente, separa o sobrenome do restante do nome utilizando `split(", ")`.
    - Em seguida, separa o título do primeiro nome utilizando `split(".")`.
4. Retorna a parte da string que corresponde ao título (pronome de tratamento).
5. Se a string não contém os padrões esperados, a função retorna "None".

In [ ]:
def get_pronouns(name) -> str:
    if ", " in name and "." in name:
        return name.split(", ")[1].split(".")[0]
    return "None" 

**Descrição**:
A função `encoding_pronouns` realiza a codificação dos pronomes de tratamento presentes na coluna 'Pronouns' de um DataFrame, mapeando os valores categóricos para valores inteiros.

**Parâmetros**:
- `df` (pd.DataFrame): O DataFrame contendo a coluna 'Pronouns' que será codificada.

**Retorno**:
- Retorna o DataFrame com a coluna 'Pronouns' codificada em valores inteiros.

**Funcionamento**:
1. A função recebe um DataFrame (`df`) que contém a coluna 'Pronouns'.
2. Utiliza o método `map` para converter os pronomes de tratamento em valores inteiros de acordo com o seguinte mapeamento:
    - 'Mrs': 5
    - 'Miss': 4
    - 'Master': 3
    - 'Other': 2
    - 'Mr': 1
3. Utiliza o método `fillna` para preencher valores nulos com os valores originais da coluna 'Pronouns', caso haja pronomes não incluídos no mapeamento.
4. Retorna o DataFrame com a coluna 'Pronouns' atualizada.

In [ ]:
def encoding_pronouns(df: pd.DataFrame) -> pd.DataFrame:
    df.Pronouns = df.Pronouns.map({'Mrs': 5, 'Miss': 4, 'Master': 3, 'Other': 2, 'Mr': 1}).fillna(df.Pronouns)
    # df = pd.get_dummies(data=df, columns=['Pronouns'], dtype=int)
    return df

**Descrição**:
A função `join_pronouns` mapeia e une diversos pronomes de tratamento em categorias padronizadas dentro de um DataFrame, substituindo pronomes específicos por categorias mais gerais.

**Parâmetros**:
- `df` (pd.DataFrame): O DataFrame contendo a coluna 'Pronouns' que será mapeada e unificada.

**Retorno**:
- Retorna o DataFrame com a coluna 'Pronouns' atualizada, contendo os pronomes padronizados.

**Funcionamento**:
1. A função recebe um DataFrame (`df`) que contém a coluna 'Pronouns'.
2. Utiliza o método `map` para substituir os pronomes de tratamento 'Mlle', 'Ms', e 'Mme' por 'Miss' e 'Mrs', respectivamente, preenchendo valores nulos com os valores originais da coluna 'Pronouns'.
3. Utiliza o método `map` novamente para substituir diversos outros pronomes (como 'Dr', 'Rev', 'Col', etc.) pela categoria 'Other', preenchendo valores nulos com os valores originais da coluna 'Pronouns'.
4. Retorna o DataFrame com a coluna 'Pronouns' atualizada e padronizada.

In [ ]:
def join_pronouns(df: pd.DataFrame) -> pd.DataFrame:
    df['Pronouns'] = df['Pronouns'].map({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'}).fillna(df['Pronouns'])
    df.Pronouns = df['Pronouns'].map({
        'Dr': 'Other', 
        'Rev': 'Other', 
        'Col': 'Other', 
        'Major': 'Other', 
        'Don': 'Other', 
        'Lady': 'Other', 
        'Sir': 'Other', 
        'Capt': 'Other', 
        'the Countess': 'Other', 
        'Jonkheer': 'Other',
        'Dona': 'Other',
    }).fillna(df['Pronouns'])
    
    return df

**Descrição**:
A função `process_embarked` preenche valores nulos na coluna 'Embarked' de um DataFrame com o valor mais frequente e, em seguida, codifica os valores da coluna em valores numéricos.

**Parâmetros**:
- `df` (pd.DataFrame): O DataFrame contendo a coluna 'Embarked' que será processada.

**Retorno**:
- Retorna o DataFrame com a coluna 'Embarked' sem valores nulos e com os valores originais codificados em inteiros.

**Funcionamento**:
1. A função recebe um DataFrame (`df`) que contém a coluna 'Embarked'.
2. Preenche os valores nulos na coluna 'Embarked' com o valor mais frequente (modo) da coluna.
3. Utiliza `LabelEncoder` para transformar os valores categóricos da coluna 'Embarked' em valores numéricos.
4. Retorna o DataFrame com a coluna 'Embarked' atualizada.

In [ ]:
def process_embarked(df: pd.DataFrame) -> pd.DataFrame:
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])    
    df.Embarked = LabelEncoder().fit_transform(df['Embarked'])
    return df

**Descrição**:
A função `process_age` preenche valores nulos na coluna 'Age' de um DataFrame com a média das idades, calculada com base no sexo e na classe dos passageiros.

**Parâmetros**:
- `df` (pd.DataFrame): O DataFrame contendo a coluna 'Age' que será processada.

**Retorno**:
- Retorna o DataFrame com a coluna 'Age' sem valores nulos, onde os valores ausentes foram preenchidos com a média das idades para o mesmo sexo e classe.

**Funcionamento**:
1. A função recebe um DataFrame (`df`) que contém a coluna 'Age'.
2. Utiliza o método `fillna` para preencher os valores nulos na coluna 'Age' com a média das idades dos passageiros que pertencem ao mesmo grupo de sexo (`Sex`) e classe (`Pclass`).
3. O método `groupby` agrupa os dados por sexo e classe, e o método `transform('mean')` calcula a média da idade dentro de cada grupo.
4. Retorna o DataFrame com a coluna 'Age' atualizada.

In [ ]:
def process_age(df: pd.DataFrame):
    df['Age'] = df['Age'].fillna(df.groupby(by=['Sex', 'Pclass'])['Age'].transform('mean'))
    return df

**Descrição**:
A função `calcular_roc_auc` calcula a curva ROC e a área sob a curva (AUC) para um conjunto de valores verdadeiros e previstos.

**Parâmetros**:
- `y_true`: Lista ou array contendo os valores verdadeiros das classes.
- `y_pred`: Lista ou array contendo as previsões do modelo.

**Retorno**:
- Retorna o valor da AUC (Área Sob a Curva ROC).

**Funcionamento**:
1. A função recebe dois parâmetros: `y_true` com os valores verdadeiros das classes e `y_pred` com as previsões do modelo.
2. Utiliza a função `roc_curve` para calcular a curva ROC (taxa de falsos positivos `fpr` e taxa de verdadeiros positivos `tpr`).
3. Calcula a área sob a curva (AUC) utilizando a função `auc` aplicada aos valores `fpr` e `tpr`.
4. Retorna o valor calculado da AUC.

In [ ]:
def calcular_roc_auc(y_true, y_pred):
    # Calcular a curva ROC
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    
    # Calcular a área sob a curva (AUC)
    roc_auc = auc(fpr, tpr)
    
    return roc_auc

In [ ]:
def exibir_metricas_desempenho(ypred, ytrue):
    acuracia = accuracy_score(y_pred=ypred, y_true=ytrue)
    precisao = precision_score(y_pred=ypred, y_true=ytrue)
    revocacao = recall_score(y_pred=ypred, y_true=ytrue)
    roc_auc = calcular_roc_auc(y_pred=ypred, y_true=ytrue)

    print(f'Acurácia: {round(acuracia, 5)}')
    print(f'Precisão: {round(precisao, 5)}')
    print(f'Revocação: {round(revocacao, 5)}')
    print(f'ROC-AUC: {round(roc_auc, 5)}')

In [ ]:
def plot_roc_auc(ytrue, ypred):
    # Gerar curva de precisão-revocação
    precision, recall, _ = precision_recall_curve(ytrue, ypred)

    plt.figure(figsize=(14, 7))

    # Subplot para a curva de precisão-revocação
    plt.subplot(1, 2, 1)
    plt.plot(recall, precision, marker='.', label='Precision-Recall Curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()

    # Subplot para a curva ROC
    fpr, tpr, _ = roc_curve(ytrue, ypred)

    plt.subplot(1, 2, 2)
    plt.plot(fpr, tpr, marker='.', label='ROC Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()

    # Mostrar os gráficos
    plt.show()

### **<span style="color:green">OBTENHA E PROCESSE OS DADOS</span>**

In [ ]:
def obter_dataframe_processado(path: str):
    df = carregar_dados(path=path)
    df = process_age(df=df)
    df = adicionar_indicador_menor_idade(df)
    df = transformar_sex_em_inteiro(df)
    df['Pronouns'] = df.Name.map(get_pronouns)
    df = join_pronouns(df)
    df = encoding_pronouns(df=df)
    df = process_embarked(df=df)
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    return df 

Obtenha o dataframe utilizando o método ``obter_dataframe_processado()`` e visualize as transformações com a função head()

In [ ]:
df = 

### **<span id="id_secao_prevendo_sobreviventes" style="color:green">SEPARE OS DADOS DE TREINO E TESTE</span>**

Reutilize o método `obter_treino_e_teste()` para dividir o dataframe em treino e teste.

In [ ]:
treino, teste = obter_treino_e_teste()

In [ ]:
cols_to_predict = ['Pclass', 'Fare', 'SibSp', 'Parch', 'Sex', 'isChild', 'Embarked', 'Age', 'Pronouns']

Crie x_treino, y_treino, x_teste e y_teste utilizando os dataframes de trino e teste.

In [ ]:
x_treino = 
y_treino = 

x_teste = 
y_teste = 

### **<span id="id_secao_prevendo_sobreviventes" style="color:green">AVALIE DIVERSOS MODELOS EM VALIDAÇÃO CRUZADA</span>**

#### Crie um array de tuplas para armazenar os modelos que serão processados em validação cruzada

##### Razões para Padronizar Dados em Modelos de Machine Learning

1. **Consistência de Escala**
Padronizar dados garante que todas as variáveis estejam na mesma escala, o que é crucial para algoritmos de Machine Learning que dependem da distância entre pontos de dados, como ***regressão linear e k-vizinhos mais próximos***.

2. **Melhoria na Convergência**
Algoritmos de otimização, como gradiente descendente, convergem mais rápido e com mais eficiência quando os dados são padronizados.

3. **Redução de Vieses**
A padronização ajuda a reduzir vieses que podem surgir devido a diferenças de escala, permitindo que o modelo trate todas as variáveis de forma justa.

4. **Melhor Desempenho de Modelos**
Modelos como Support Vector Machines ***(SVM) e Redes Neurais*** beneficiam-se da padronização dos dados, levando a um desempenho superior e resultados mais precisos.

Conclusão
Padronizar dados é um passo fundamental no pré-processamento de dados para Machine Learning, garantindo a consistência e eficácia dos modelos treinados. Certifique-se de incluir esse passo no seu pipeline de dados para obter melhores resultados nos seus projetos de Machine Learning.

---


Crie um array de tuplas no formato ('aliasModelo', modelo), para armazenar modelos que tem melhora no desempenho se receber dados padronizados no processo de treinamento e teste. Inicialmente, utilizaremos os modelos ***XGBClassifier, LogisticRegression, SGDClassifier, KNeighborsClassifier***.

In [ ]:
# Modelos que precisam de padronização
scaled_models = [

]

Faça o mesmo com os modelos ***GradientBoostingClassifier, LGBMClassifier, RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier, RidgeClassifier, DecisionTreeClassifier***. 

In [ ]:
# Modelos sem necessidade de padronização
non_scaled_models = [
    
]

**Descrição**:
O código aplica o `StandardScaler` apenas aos modelos que são sensíveis à escala dos dados, utilizando um pipeline que sequencia a padronização dos dados seguida pelo treinamento do modelo.

**Funcionamento do Código**:

1. **Definição dos Modelos Sensíveis à Escala**:
    - Os modelos são definidos em uma lista chamada `scaled_models`, onde cada elemento é uma tupla que contém o nome do modelo e a instância do modelo. Estes modelos são conhecidos por se beneficiarem da padronização dos dados.

2. **Criação do Pipeline**:
    - Utiliza-se uma compreensão de lista para iterar sobre cada tupla na lista `scaled_models`.
    - Para cada tupla (nome, modelo), cria-se um `Pipeline` do `scikit-learn` que consiste em dois estágios:
        1. `'scaler'`: Aplica o `StandardScaler` para padronizar os dados.
        2. `'classifier'`: Aplica o modelo de classificação.
    - O `Pipeline` é então empacotado novamente em uma tupla contendo o nome do modelo e o pipeline completo.

3. **Lista de Pipelines**:
    - A lista `pipelines` resultante contém todas as tuplas de (nome, pipeline) para os modelos sensíveis à escala.

Crie e aplique um pipeline sobre os models sensíveis à escala (scaled_models). 

Mas o que é um **pipeline**? Um pipeline é uma ferramenta poderosa que permite concatenar várias etapas de processamento de dados e modelagem em uma única sequência.

In [ ]:
# Aplicando o Pipeline com StandardScaler apenas aos modelos sensíveis à escala


In [ ]:
all_models = non_scaled_models + pipelines

---

**Descrição**:
A função `avaliar_modelos` realiza a validação cruzada em diversos modelos de Machine Learning e retorna a média e o desvio padrão das pontuações de precisão.

**Parâmetros**:
- `modelos`: Uma lista de tuplas contendo o nome do modelo e a instância do modelo.
- `X`: O DataFrame contendo as características (features) do dataset.
- `y`: A série ou DataFrame contendo os rótulos (labels) do dataset.
- `n_splits` (int, opcional): Número de folds para a validação cruzada (padrão é 10).
- `metric` (str, opcional): Métrica de avaliação dos modelos (padrão é 'accuracy').

**Retorno**:
- Retorna um dicionário `resultados` onde as chaves são os nomes dos modelos e os valores são dicionários contendo a média e o desvio padrão das pontuações de precisão.

**Funcionamento**:
1. A função inicializa um dicionário `resultados` para armazenar os resultados de cada modelo.
2. Utiliza `KFold` do `sklearn` para dividir o dataset em `n_splits` folds, com embaralhamento dos dados e uma semente aleatória para reprodutibilidade.
3. Itera sobre cada modelo na lista `modelos`:
    - Para cada modelo, inicializa uma lista `scores` para armazenar as pontuações de cada fold.
    - Realiza a validação cruzada sem estratificação:
        - Divide os dados de treinamento e teste para cada fold.
        - Treina o modelo com os dados de treinamento.
        - Prediz os rótulos para os dados de teste.
        - Calcula e armazena a pontuação de precisão para o fold atual.
    - Calcula a média e o desvio padrão das pontuações de precisão e armazena no dicionário `resultados` com o nome do modelo como chave.
4. Retorna o dicionário `resultados`.

**Exemplo de Uso**:
```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

modelos = [
    ('Random Forest', RandomForestClassifier()),
    ('Logistic Regression', LogisticRegression())
]

resultados = avaliar_modelos(modelos, X, y)
print(resultados)


---

Crie uma função para avaliar modelos em validação cruzada.

In [ ]:
from sklearn.model_selection import KFold

def avaliar_modelos(modelos, X, y, n_splits=10, metric='accuracy'):
    resultados = {}
    
    # Divide a base de dados em 10 folds. 
    kf = KFold()
    
    for nome, modelo in modelos:
        scores = []
        
        # Realizar a validação cruzada sem estratificação
        for train_idx, test_idx in kf.split(X):
            X_train, X_test = 
            y_train, y_test = 
            
            modelo.fit()
            y_pred = modelo.predict()
            scores.append(accuracy_score())
        
        resultados[nome] = {
            'media': ,
            'desvio_padrao': 
        }
    
    return resultados


---

Visualize o desempenho dos modelos percorrendo o dicionário de resultados devolvidos pela função ``avaliar_modelos()``

In [ ]:
resultados = avaliar_modelos()

for nome, resultado in resultados.items():
    print(f"{nome}: Média = {resultado['media']:.5f}, Desvio Padrão = {resultado['desvio_padrao']:.4f}")

### **<span style="color:green">COMBINE DIVERSOS MODELOS EM UM ENSAMBLE E FAA PREVISÕES</span>**

In [ ]:
# Criando o StackingClassifier
stacking_model = StackingClassifier(
    estimators= ,
    final_estimator= ,  # Meta-aprendiz (pode ser ajustado)
    passthrough= ,  # Se True, inclui as features originais junto com as previsões na meta-aprendizagem
    n_jobs=-1
)

# Exemplo de treinamento
stacking_model.fit()
y_pred = stacking_model.predict()
accuracy_score()